# Schema Linking Accuracy Metric (SLAM):

### Imports

In [6]:
import pandas as pd
import ast
import json

## Creating DFs

### Function to import JSON

In [38]:
def import_json_file(file_path):
    # Load the JSON data from the file
    with open(file_path, 'r') as json_file:
        json_data = json.load(json_file)

    # Create a DataFrame from the extracted data
    df_dev_set = pd.DataFrame(json_data, columns=["question_id", "db_id", "tables"])

    #df_dev_set.head()
    return df_dev_set

### Function to import CSV

In [39]:
# Function to import a CSV file into a pandas DataFrame with the given schema
def import_csv_file(file_path):
    # Define a custom converter to parse the string representation of lists
    def parse_list(x):
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return []

    # Specify the column data types
    dtype_dict = {
        'question_id': int,
        'predicted_tables': str,
        'total_tables': int
    }

    # Specify the converters for list columns
    converters = {
        'predicted_tables': parse_list
    }

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path, dtype=dtype_dict, converters=converters)
    return df

### Function to compare tables from dev and gen

In [40]:
def compare_lists(ground_truth, predicted):

    true_positives = len(set(ground_truth) & set(predicted))
    false_positives = len(set(predicted) - set(ground_truth))
    false_negatives = len(set(ground_truth) - set(predicted))
    true_negatives = 0  # Not applicable for this scenario

    return true_positives, true_negatives, false_positives, false_negatives



### Processing

In [41]:
df1 = import_json_file('dev/dev_subset.json')
df1.head()

,question_id,db_id,tables
0,0,california_schools,[frpm]
1,1,california_schools,[frpm]
2,2,california_schools,"[frpm, schools]"
3,3,california_schools,"[frpm, schools]"
4,4,california_schools,"[frpm, schools]"


In [42]:
df2 = import_csv_file('linked_tables_json_conservative.csv')
df2.head()

/var/folders/cp/g5zclxh15qxcyn95c4jr6fd80000gn/T/ipykernel_95571/3972835819.py:23: ParserWarning: Both a converter and dtype were specified for column predicted_tables - only the converter will be used.
  df = pd.read_csv(file_path, dtype=dtype_dict, converters=converters)


,question_id,db_id,gold_tables,predicted_tables,total_tables
0,0,california_schools,['frpm'],"[frpm, schools]",3
1,1,california_schools,['frpm'],"[frpm, satscores, schools]",3
2,2,california_schools,"['frpm', 'schools']","[frpm, satscores, schools]",3
3,3,california_schools,"['frpm', 'schools']","[frpm, satscores, schools]",3
4,4,california_schools,"['frpm', 'schools']","[frpm, schools]",3


In [43]:
merged_df = df1.merge(df2, on='question_id', how='inner')
merged_df.head()



merged_df = merged_df.rename(columns={'db_id_x': 'db_id'})
merged_df = merged_df.drop('db_id_y', axis=1)

merged_df.head()
#
final_df = pd.DataFrame(columns=['question_id','db_id','tables','predicted_tables','total_tables','tp','tn','fp','fn','precision','recall','f1_score'])
final_df.head()

,question_id,db_id,tables,predicted_tables,total_tables,tp,tn,fp,fn,precision,recall,f1_score


In [44]:

for index, row in enumerate(merged_df.iterrows()):
    row = row[1]
    #row2 = row2[1]
    tp, tn, fp, fn = compare_lists(row['tables'], row['predicted_tables'])


    # Calculate precision, recall, and F1 score for each row
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if precision == 0 and recall == 0:
        f1_score = 0.0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    #print('question_id: ',row['question_id'],'|| predicted_tables:',row['predicted_tables'],'|| true_tables:',row['tables'])
    #print("True Positives:", tp)
    #print("True Negatives:", tn)
    #print("False Positives:", fp)
    #print("False Negatives:", fn)
    #print("Precision:", precision)
    #print("Recall:", recall)
    #print("F1-Score:", f1_score)
    #print('-----------------------------------')

    metrics_dict = {
        'question_id': row['question_id'],
        'db_id': row['db_id'],
        'tables': row['tables'],
        'predicted_tables': row['predicted_tables'],
        'total_tables': row['total_tables'],
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

    final_df = pd.concat([final_df, pd.DataFrame([metrics_dict])], ignore_index=True)

final_df.to_csv('metrics_result_conservative.csv', index=False)
final_df.head()

,question_id,db_id,tables,predicted_tables,total_tables,tp,tn,fp,fn,precision,recall,f1_score
0,0,california_schools,[frpm],"[frpm, schools]",3,1,0,1,0,0.500000,1.0,0.666667
1,1,california_schools,[frpm],"[frpm, satscores, schools]",3,1,0,2,0,0.333333,1.0,0.500000
2,2,california_schools,"[frpm, schools]","[frpm, satscores, schools]",3,2,0,1,0,0.666667,1.0,0.800000
3,3,california_schools,"[frpm, schools]","[frpm, satscores, schools]",3,2,0,1,0,0.666667,1.0,0.800000
4,4,california_schools,"[frpm, schools]","[frpm, schools]",3,2,0,0,0,1.000000,1.0,1.000000


In [45]:
final_df

,question_id,db_id,tables,predicted_tables,total_tables,tp,tn,fp,fn,precision,recall,f1_score
0,0,california_schools,[frpm],"[frpm, schools]",3,1,0,1,0,0.500000,1.0,0.666667
1,1,california_schools,[frpm],"[frpm, satscores, schools]",3,1,0,2,0,0.333333,1.0,0.500000
2,2,california_schools,"[frpm, schools]","[frpm, satscores, schools]",3,2,0,1,0,0.666667,1.0,0.800000
3,3,california_schools,"[frpm, schools]","[frpm, satscores, schools]",3,2,0,1,0,0.666667,1.0,0.800000
4,4,california_schools,"[frpm, schools]","[frpm, schools]",3,2,0,0,0,1.000000,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
105,1475,debit_card_specializing,"[customers, yearmonth]","[customers, transactions_1k, yearmonth]",5,2,0,1,0,0.666667,1.0,0.800000
106,1476,debit_card_specializing,"[customers, yearmonth]","[customers, transactions_1k, yearmonth]",5,2,0,1,0,0.666667,1.0,0.800000
107,1477,debit_card_specializing,"[customers, yearmonth]","[customers, gasstations, products, transaction...",5,2,0,3,0,0.400000,1.0,0.571429
108,1478,debit_card_specializing,"[customers, yearmonth]","[customers, gasstations, transactions_1k, year...",5,2,0,2,0,0.500000,1.0,0.666667


In [46]:
total_tp = final_df['tp'].sum()
total_tn = final_df['tn'].sum()
total_fp = final_df['fp'].sum()
total_fn = final_df['fn'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) != 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) != 0 else 0
overall_f1_score = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) != 0 else 0


print(f'Overall Precision: {overall_precision:.2f}')
print(f'Overall Recall: {overall_recall:.2f}')
print(f'Overall F1 Score: {overall_f1_score:.2f}')




Overall Precision: 0.56
Overall Recall: 0.98
Overall F1 Score: 0.71


In [ ]:
from azure_openai import get_embedding